<a href="https://colab.research.google.com/github/maberf/colabs/blob/main/Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ##### PORTFOLIO INVESTOR CODE #####
#
# GOOGLE SHEETS - Code to upload data in Google Sheets to support investiments decisions. The Google Sheets can be adjusting according user desire, but its location should be adjusted in code also.
# Quotes  - Google Sheets file name with all sheets inside
# Stockvar - Sheet with data with B3 brazilian stocks history series in BRL
# RealSatevar - Sheet with data with B3 brazilian REITs history series in BRL
# StockUSvar - Sheet with data with US stocks history series in USD
# ETFUSvar - Sheet with data with US ETFs history series in USD
# RealSateUSvar - Sheet with data with US REITs history series in USD
# Portfolio - Sheet with portfolio considering assets in list in BRL. US assets are converted in BRL.
# YFStockUS - Yahoo Finance provide US assets fundamentalist informations directly, but not in case o B3 assets.
# Fundamentus - In B3 case, a new code is necessary scraping informations from specialized sites such as https://www.fundamentus.com.br . This is in fundamentus python code.
# Quote - It is suggested create a Google Sheets sheet in Quotes file which consolidate all quotes and where you can import data from other sheets. It is suggested to use Google Finance in this sheets once the quototion update is faster than in code running.
# Discounted Cash Flow asset evaluations can be done directly in this Google Sheets Quote sheet.
#
# The lists with the tickers and other informations should be filled directly in the code where there are INSERT OR ADJUST HERE lines.
# IFIX - History series csv file with time desired period (2y, 3y, etc.) from https://br.investing.com/indices/bm-fbovespa-real-estate-ifix-historical-data should be downloaded to a Google Drive directory which is in the path in code.
# FUNDS EXPLORER - Copy and paste date from https://www.fundsexplorer.com.br/ranking in a Google Drive directory excel file which is in the path in code. It is necessary to SELECT ALL COLUMNS in Funds Explorer website.
# Before to run the code it is necessary to make these previous activities and insert the input data . Check the code below until the ENDING POINT TO MANUAL ACTIVITIES.
# Verifiy data on thes files in case of code running problems.
# During the code running it is necessary to allow access to Google. Necessary code access Google Drive where files are located and Google Sheets to upload the results.

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import yfinance as yf
import os
import datetime as dt
from google.colab import drive
from google.colab import auth
from google.auth import default
import gspread

In [ ]:
# Stock tickers - Insert here# Real Estate Invesments Trust tickers - INSERT OR ADJUST HERE
# Try to mantain IBOV + 30 assets in alphabetical order to easly adjust google docs spreadsheet
tickers = ['^BVSP','ABEV3.SA','BBAS3.SA','BBDC4.SA','BBSE3.SA','CMIN3.SA','CPFE3.SA','CPLE6.SA','ELET6.SA','FLRY3.SA','ITUB4.SA','IVVB11.SA','JHSF3.SA','KLBN11.SA','MBRF3.SA','LEVE3.SA','LREN3.SA','NEOE3.SA','PETR4.SA','POMO4.SA','QQQI11.SA','RECV3.SA','RENT3.SA','SBSP3.SA','TGMA3.SA','TIMS3.SA','TUPY3.SA','VALE3.SA','VIVA3.SA','VIVT3.SA','WEGE3.SA']

In [ ]:
# Real Estate Investments Trust tickers - INSERT OR ADJUST HERE
# Try to mantain IFIX + 20 assets in alphabetical order to easly adjust google docs spreadsheet
# IFIX here only to a space in dataframe, yahoo finance return IFIX.SA maximum 5 days. IFIX should be calculated downloading an Investing.com csv file history and it must be uploaded in personal Google Drive. This code read it.
tickerr = ['IFIX.SA','BTLG11.SA','HGCR11.SA','HGBS11.SA','HGRE11.SA','HGRU11.SA','HSLG11.SA','HSML11.SA','HTMX11.SA','JSAF11.SA','JFLL11.SA','KNCA11.SA','KNHF11.SA','KNIP11.SA','MXRF11.SA','MFII11.SA','SAPI11.SA','TGAR11.SA','TRXF11.SA','VGHF11.SA','VISC11.SA']

In [ ]:
# US Stocks tickers - INSERT OR ADJUST HERE
# Try to mantain SP500 + USDBRL + 30 assets in alphabetical order to easly adjust google docs spreadsheet
tickersus = ['^GSPC','USDBRL=X','AAPL','AMZN','ASML','BAC','BKNG','BMY','BRK-B','CRWD','EXC','GOOG','HALO','JNJ','JPM','KMB','KO','LLY','META','MRK','MSFT','NOW','NVDA','PFE','PLTR','RIO','T','TSLA','TSM','V','XOM','WMT']

In [ ]:
# US ETFs tickers - INSERT OR ADJUST HERE
# Try to mantain SP500 + USDBRL + 10 assets in alphabetical order to easly adjust google docs spreadsheet
tickereus = ['^GSPC','USDBRL=X','FBTC','JEPI','HACK', 'IVV','SCHD','SOXX','SPY','TLT','SMH','TFLO']

In [ ]:
# US ETFs REITS tickers - INSERT OR ADJUST HERE
# Try to mantain SP500 + USDBRL + 10 assets in alphabetical order to easly adjust google docs spreadsheet
tickerrus = ['USRT','USDBRL=X','O','STAG','ADC', 'LTC','SLG','WELL','PLD','AMT','EQIX','SPG']

In [ ]:
# Portfolio tickers - INSERT OR ADJUST HERE
tickerport = ['^BVSP','USDBRL=X','AMZN','BBSE3.SA','BRK-B','CPFE3.SA','ELET6.SA','FBTC','HTMX11.SA','JHSF3.SA','JSAF11.SA','KMB','MRK','MSFT','PETR4.SA','POMO4.SA','SAPI11.SA','SCHD','TFLO','TGAR11.SA','TRXF11.SA','VALE3.SA']
# Portfolio tickers weight - INSERT OR ADJUST HERE IN THE SAME ORDER!
weightport = [0.000, 0.000, 5.081, 7.509, 10.812, 0.000, 8.292, 2.714, 5.311, 0.000, 6.269, 3.388, 3.388, 3.388, 13.885, 0.000, 6.410, 1.355, 3.389, 4.908, 13.882, 0.000]
# Portfolio tickers expected returns - INSERT OR ADJUST HERE IN THE SAME ORDER!
# IBOV (^BVSP) and USD (USDBRL=X) MUST always be zero!
expretport = [0.0, 8.0, 23.9, 18.5, 18.5, 2.7, 6.0, 25.9, 20.5, 25.3, 28.1, 19.8, 24.6, 21.5, 28.4, 13.8, 17.1, 8.6, 8.6, 22.1, 19.4, 5.8]
# Portfolio assets limits - INSERT OR ADJUST HERE IN THE SAME ORDER!
# IBOV (^BVSP) and USD (USDBRL=X) MUST always be zero!
limitsassetsport = [0.0, 0.0, 10.0, 10.0, 10.0, 10.0, 5.0, 5.0, 10.0, 5.0, 6.0, 5.0, 5.0, 5.0, 15.0, 5.0, 6.0, 5.0, 5.0, 5.0, 15.0, 10.0]
fatorcorr = 0.90987 # CORRECTION FACTOR to adjusts weights just to variable incomes assets discounting other ones such as risk free incomes. It is to simplify comparision directly in your investment spreadsheet. If it is only varible incomes, correction factor is 1.0.
limitsassetsport = [x * fatorcorr for x in limitsassetsport]
# Print just to verify lists lengths
print(len(tickerport))
print(len(weightport))
print(len(expretport))
print(len(limitsassetsport))


In [ ]:
# Risk free rate BRAZIL in percentage - INSERT OR ADJUST HERE
riskfree = 13.88

In [ ]:
# Target Sharpe in percentage of Sharpe Maximum - INSERT OR ADJUST HERE
target_per = 85.0

In [ ]:
# Portfolio dataframe creation
portfolio = pd.DataFrame({'Ticker': tickerport, 'W': weightport, 'RetE%':expretport})

In [ ]:
# Excluding .SA, renaming ^BVSP to IBOV and USDBRL=X to USDBRL
portfolio['Ticker'] = portfolio['Ticker'].str.replace('.SA', '', regex=False)
portfolio['Ticker'] = portfolio['Ticker'].str.replace('^BVSP', 'IBOV', regex=False)
portfolio['Ticker'] = portfolio['Ticker'].str.replace('USDBRL=X', 'USDBRL', regex=False)
# display(portfolio)

In [ ]:
# Load tickers history prices in a dataframe considering a certain period of time - ADJUST HERE, default 1 year (1y)
# Sometimes some ticker has problems in yahoo finance. If it happens, close the session and try again. Or change the ticker, because problem in one ticker will cause problem in all code running.
# Check success download completed to all dataframes, otherwise the code will broke in next lines.
dfs = yf.download(tickers, period='2y', auto_adjust=True)['Close']
dfr = yf.download(tickerr, period='2y', auto_adjust=True)['Close']
dfsus = yf.download(tickersus, period='2y', auto_adjust=True)['Close']
dfeus = yf.download(tickereus, period='2y', auto_adjust=True)['Close']
dfrus = yf.download(tickerrus, period='2y', auto_adjust=True)['Close']
dfport = yf.download(tickerport, period='2y', auto_adjust=True)['Close']
# Remove timezone from index
dfs.index = pd.to_datetime(dfs.index).tz_localize(None)
dfr.index = pd.to_datetime(dfr.index).tz_localize(None)
dfsus.index = pd.to_datetime(dfsus.index).tz_localize(None)
dfeus.index = pd.to_datetime(dfeus.index).tz_localize(None)
dfrus.index = pd.to_datetime(dfrus.index).tz_localize(None)
dfport.index = pd.to_datetime(dfport.index).tz_localize(None)
# display(dfs)
# display(dfr)
# display(dfeus)
# display(dfrus)
# display(dfport)

In [ ]:
# IFIX historic series from Investing.com to be appended in real state dataframe dfr - https://br.investing.com/indices/bm-fbovespa-real-estate-ifix-historical-data - MAKE THIS ACTIVITY
# Download the file from site and copy to your google drive. Rename de file as history.csv. Adjust the path below in " ifixfile = .... " command line according your file location.
# Google Drive mounth
drive.mount('/content/drive', force_remount=True)
# File path on Google Drive - Download the file and upload to Financas folder in Google Drive. Rename the path according file name uploaded.
ifixfile = '/content/drive/MyDrive/Financas/history.csv'
# File csv to dataframe converting quote to float
ifix = pd.read_csv(ifixfile, thousands = '.', decimal = ',', dtype = {'Último':np.float64})
# Excluding and rename columns
ifix = ifix.drop(columns=['Abertura', 'Máxima', 'Mínima', 'Vol.', 'Var%'])
ifix = ifix.rename(columns={'Data': 'Date', 'Último': 'IFIX.SA'})
# Date format in Date column
ifix['Date'] = pd.to_datetime(ifix['Date'], format='%d%m%Y', errors='coerce')
ifix.set_index('Date', inplace=True)
# Solve eventual duplicated registers, grouped by mean
ifix = ifix.groupby(level=0).mean()
# display(ifix)

In [ ]:
# Go to Funds Explorer Table in https://www.fundsexplorer.com.br/ranking  - MAKE THIS ACTIVITY
# Manually select all columns in web page and paste all data AS VALUES ONLY in a NEW Excel File sheet.
# Save the file with the name fundsexplorer.xlsx in a folder called Financas (or in another preferable path, adjusting the path below)
fundsexplorerfile = '/content/drive/MyDrive/Financas/fundsexplorer.xlsx'
# Excel file reading
fundsexplorer = pd.read_excel(fundsexplorerfile)
#display(fundsexplorer)
# The sequence of FUNDS EXPLORER ROUTINE, see at the end of the code.

In [ ]:
# ENDING POINT TO MANUAL ACTIVITIES. The code should run after this point. Permission to Google Drive access can be required during code running.

In [ ]:
# Replace dfr dataframe by ifix values by index key (date)
dfr.update(ifix)
# display(dfr)

In [ ]:
# Excluding .SA, renaming ^BVSP to IBOV, ^GSPC to SP500
dfs.columns = [col.replace('.SA', '') for col in dfs.columns]
dfs.columns = [col.replace('^BVSP', 'IBOV') for col in dfs.columns]
dfr.columns = [col.replace('.SA', '') for col in dfr.columns]
dfsus.columns = [col.replace('^GSPC', 'SP500') for col in dfsus.columns]
dfsus.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfsus.columns]
dfeus.columns = [col.replace('^GSPC', 'SP500') for col in dfeus.columns]
dfeus.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfeus.columns]
# dfrus.columns = [col.replace('^GSPC', 'SP500') for col in dfrus.columns]
dfrus.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfrus.columns]
dfport.columns = [col.replace('.SA', '') for col in dfport.columns]
dfport.columns = [col.replace('^BVSP', 'IBOV') for col in dfport.columns]
dfport.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfport.columns]

In [ ]:
# Other conformations such as ascending order and the market indexes in the first column
dfs = dfs[sorted(dfs.columns)]
dfr = dfr[sorted(dfr.columns)]
dfsus = dfsus[sorted(dfsus.columns)]
dfeus = dfeus[sorted(dfeus.columns)]
dfrus = dfrus[sorted(dfrus.columns)]
dfport = dfport[sorted(dfport.columns)]
dfs = dfs[['IBOV'] + [col for col in dfs.columns if col != 'IBOV']]
dfr = dfr[['IFIX'] + [col for col in dfr.columns if col != 'IFIX']]
dfsus = dfsus[['SP500', 'USDBRL'] + [col for col in dfsus.columns if col not in ['SP500', 'USDBRL']]]
dfeus = dfeus[['SP500', 'USDBRL'] + [col for col in dfeus.columns if col not in ['SP500', 'USDBRL']]]
dfrus = dfrus[['USRT', 'USDBRL'] + [col for col in dfrus.columns if col not in ['USRT', 'USDBRL']]]
dfport = dfport[['IBOV', 'USDBRL'] + [col for col in dfport.columns if col not in ['IBOV', 'USDBRL']]]
# Here dataframes should be ready for calculations. It will be made and uploaded in dfxvar dataframes later.
# display(dfs)
# display(dfr)
# display(dfsus)
# display(dfeus)
# display(dfrus)
# display(dfport)

In [ ]:
# Timeline harmonization, same period of time
common_idx = (
    dfs.index
    .intersection(dfr.index)
    .intersection(dfsus.index)
    .intersection(dfeus.index)
    .intersection(dfrus.index)
    .intersection(dfport.index)
)

# Reindex e preencher (sem warnings)
dfs = dfs.reindex(common_idx).ffill()
dfr = dfr.reindex(common_idx).ffill()
dfsus = dfsus.reindex(common_idx).ffill()
dfeus = dfeus.reindex(common_idx).ffill()
dfrus = dfrus.reindex(common_idx).ffill()
dfport = dfport.reindex(common_idx).ffill()


In [ ]:
# Portfolio daframe in BRL
# Multiply USA assets by USDBRL value (BRL quotation)
#
usd_sources = []
for name in ('dfsus', 'dfeus', 'dfrus'):
    if name in globals():
        try:
            df_src = globals()[name]
            # consider only columns (tickers) of reference dataframe
            usd_sources.extend(list(df_src.columns))
        except Exception:
            pass

usd_set = set(usd_sources)

# Identifies dfport columns that correspond to assets in USD (ignores the exchange rate column).
cols_a_converter = [c for c in dfport.columns if c != 'USDBRL' and c in usd_set]

# Performs the conversion (multiplies the price in USD by the USDBRL exchange rate line by line)
for col in cols_a_converter:
    # multiplication aligns indices; preserves NaNs where they exist.
    dfport[col] = dfport[col].multiply(dfport['USDBRL'], axis=0)

# Displayed result: which columns were converted and initial preview of dfport.
if cols_a_converter:
    print(f"Convertidas para BRL (multiplicadas por dfport['USDBRL']): {cols_a_converter}")
else:
    print("Nenhuma coluna de dfport foi identificada como cotada em USD (nenhuma conversão aplicada).")

# Show the first lines of the dfport for review.
display(dfport.head())

In [ ]:
# Calculate daily variation
dfsvar = dfs.pct_change()
dfrvar = dfr.pct_change()
dfsusvar = dfsus.pct_change()
dfeusvar = dfeus.pct_change()
dfrusvar = dfrus.pct_change()
dfportvar = dfport.pct_change()
# display(dfsvar)
# display(dfrvar)
# display(dfsusvar)
# display(dfeusvar)
# display(dfeusvar)
# display(dfportvar)

In [ ]:
# Market Percentage Historic Return calculation and column add in output dataframes
# CAGR – Compound Annual Growth Rate - calculation
#
def cagr_from_prices(series):
    s = series.dropna()
    if len(s) < 2:
        return np.nan
    total_return = s.iloc[-1] / s.iloc[0] - 1
    n_days = len(s) - 1
    # annualizes to 252 working days per year.
    return (1 + total_return) ** (252 / n_days) - 1
#
# Stocks - Applies by column (each column = one ticker)
cagr_series = dfs.apply(cagr_from_prices)  # Returns a fraction, e.g., 0.069 = 6.9%
stockvar = cagr_series.mul(100).to_frame(name='RetH%')  # converts to %
stockvar.index.name = 'Ticker'
stockvar['RetH%'] = stockvar['RetH%'].round(1)
# display(stockvar)
#
# Real State - Applies by column (each column = one ticker)
cagr_series = dfr.apply(cagr_from_prices)  # Returns a fraction, e.g., 0.069 = 6.9%
realstatevar = cagr_series.mul(100).to_frame(name='RetH%')  # converts to %
realstatevar.index.name = 'Ticker'
realstatevar['RetH%'] = realstatevar['RetH%'].round(1)
# display(realstatevar)
#
# Stock US - Applies by column (each column = one ticker)
cagr_series = dfsus.apply(cagr_from_prices)  # Returns a fraction, e.g., 0.069 = 6.9%
stockusvar = cagr_series.mul(100).to_frame(name='RetH%')  # converts to %
stockusvar.index.name = 'Ticker'
stockusvar['RetH%'] = stockusvar['RetH%'].round(1)
# display(stockusvar)
#
# ETF US - Applies by column (each column = one ticker)
cagr_series = dfeus.apply(cagr_from_prices)  # Returns a fraction, e.g., 0.069 = 6.9%
etfusvar = cagr_series.mul(100).to_frame(name='RetH%')  # converts to %
etfusvar.index.name = 'Ticker'
etfusvar['RetH%'] = etfusvar['RetH%'].round(1)
# display(etfusvar)
#
# ETF REITS US - Applies by column (each column = one ticker)
cagr_series = dfrus.apply(cagr_from_prices)  # Returns a fraction, e.g., 0.069 = 6.9%
reitusvar = cagr_series.mul(100).to_frame(name='RetH%')  # converts to %
reitusvar.index.name = 'Ticker'
reitusvar['RetH%'] = reitusvar['RetH%'].round(1)
# display(reitusvar)
#
# Portfolio - Aplica por coluna (cada coluna = um ticker)
cagr_series = dfport.apply(cagr_from_prices)  # Returns a fraction, e.g., 0.069 = 6.9%
portvar = cagr_series.mul(100).to_frame(name='RetH%')  # converts to %
portvar.index.name = 'Ticker'
portvar['RetH%'] = portvar['RetH%'].round(1)
# display(portvar)

In [ ]:
# Market return variance calculation
vars = dfsvar.var()*252
varr = dfrvar.var()*252
varsus = dfsusvar.var()*252
vareus = dfeusvar.var()*252
varrus = dfrusvar.var()*252
varport = dfportvar.var()*252
# display(vars)
# display(varr)
# display(varsus)
# display(vareus)
# display(varrus)
# display(varport)

In [ ]:
# Market risk calculation, in percentage (%). Add column in output dataframes
stockvar['Risk%'] = dfsvar.std()*np.sqrt(252)*100
stockvar['Risk%'] = stockvar['Risk%'].round(0)
realstatevar['Risk%'] = dfrvar.std()*np.sqrt(252)*100
realstatevar['Risk%'] = realstatevar['Risk%'].round(0)
stockusvar['Risk%'] = dfsusvar.std()*np.sqrt(252)*100
stockusvar['Risk%'] = stockusvar['Risk%'].round(0)
etfusvar['Risk%'] = dfeusvar.std()*np.sqrt(252)*100
etfusvar['Risk%'] = etfusvar['Risk%'].round(0)
reitusvar['Risk%'] = dfrusvar.std()*np.sqrt(252)*100
reitusvar['Risk%'] = reitusvar['Risk%'].round(0)
portvar['Risk%'] = dfportvar.std()*np.sqrt(252)*100
portvar['Risk%'] = portvar['Risk%'].round(0)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(reitusvar)
# display(portvar)

In [ ]:
# Covariance calculation
covs = dfsvar.cov()*252
covr = dfrvar.cov()*252
covsus = dfsusvar.cov()*252
coveus = dfeusvar.cov()*252
covrus = dfrusvar.cov()*252
covport = dfportvar.cov()*252
# display(covs)
# display(covr)
# display(covsus)
# display(coveus)
# display(covrus)
# display(covport)

In [ ]:
# Beta calculation
betas = covs['IBOV']/vars['IBOV']
betas = betas.round(3)
betas.name = 'Beta'
betar = covr['IFIX']/varr['IFIX']
betar = betar.round(3)
betar.name = 'Beta'
betasus = covsus['SP500']/varsus['SP500']
betasus = betasus.round(3)
betasus.name = 'Beta'
betaeus = coveus['SP500']/vareus['SP500']
betaeus = betaeus.round(3)
betaeus.name = 'Beta'
betarus = covrus['USRT']/varrus['USRT']
betarus = betarus.round(3)
betarus.name = 'Beta'
betaport = covport['IBOV']/varport['IBOV']
betaport = betaport.round(3)
betaport.name = 'Beta'
# display(betas)
# display(betar)
# display(betasus)
# display(betaeus)
# display(betarus)
# display(betaport)

In [ ]:
# Adding Beta to column output dataframes
stockvar['Beta'] = betas
realstatevar['Beta'] = betar
stockusvar['Beta'] = betasus
etfusvar['Beta'] = betaeus
reitusvar['Beta'] = betarus
portvar['Beta'] = betaport
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(reitusvar)
# display(portvar)

In [ ]:
# Adding Min to column output dataframes
stockvar['Min'] = dfs.min()
stockvar['Min'] = stockvar['Min'].round(2)
realstatevar['Min'] = dfr.min()
realstatevar['Min'] = realstatevar['Min'].round(2)
stockusvar['Min'] = dfsus.min()
stockusvar['Min'] = stockusvar['Min'].round(2)
etfusvar['Min'] = dfeus.min()
etfusvar['Min'] = etfusvar['Min'].round(2)
reitusvar['Min'] = dfrus.min()
reitusvar['Min'] = reitusvar['Min'].round(2)
portvar['Min'] = dfport.min()
portvar['Min'] = portvar['Min'].round(2)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(reitusvar)
# display(portvar)

In [ ]:
# Adding Max to column output dataframes
stockvar['Max'] = dfs.max()
stockvar['Max'] = stockvar['Max'].round(2)
realstatevar['Max'] = dfr.max()
realstatevar['Max'] = realstatevar['Max'].round(2)
stockusvar['Max'] = dfsus.max()
stockusvar['Max'] = stockusvar['Max'].round(2)
etfusvar['Max'] = dfeus.max()
etfusvar['Max'] = etfusvar['Max'].round(2)
reitusvar['Max'] = dfrus.max()
reitusvar['Max'] = reitusvar['Max'].round(2)
portvar['Max'] = dfport.max()
portvar['Max'] = portvar['Max'].round(2)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(reitusvar)
# display(portvar)

In [ ]:
# Organizing columns order and making Index column as index of dataframes
stockvar = stockvar.reset_index()
realstatevar = realstatevar.reset_index()
stockusvar = stockusvar.reset_index()
etfusvar = etfusvar.reset_index()
reitusvar = reitusvar.reset_index()
portvar = portvar.reset_index()
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(reitusvar)
# display(portvar)

In [ ]:
# FUNCTIONS TO CALCULATE PORTFOLIO PERFORMANCE, MAXIMUM SHARPE AND TARGET SHARPE
# --- Usage example ---
# Assuming you have df_returns (DataFrame) with log or simple returns, calculated daily/monthly:
# mu = df_returns.mean().values           # average return per asset (compatible frequency)
# cov = df_returns.cov().values           # covariance matrix
# risk_free = 0.0                         # adjust according to your frequency (e.g., annual rate/252 for daily)
#
# res_max, w_max = max_sharpe_weights(mu, cov, riskfree)
# print("Max Sharpe:", portfolio_performance(w_max, mu, cov, risk_free)[2], "weights:", w_max)
#
# target = 1.2
# res_tgt, w_tgt, info = weights_for_target_sharpe(mu, cov, target, riskfree)
# print("Target result:", info)
# print("Weights:", w_tgt)

In [ ]:
# Portfolio dataframe assembling
portfolio = pd.merge(portfolio, portvar, on='Ticker', how='inner')
# firstsnames = ['IBOV','USDBRL']
firstsnames = ['IBOV', 'USDBRL']
portfolio.iloc[2:] = portfolio[~portfolio['Ticker'].isin(firstsnames)].sort_values(by='Ticker').values
# display(portfolio)

In [ ]:
# Portfolio PERFORMANCE calculation function
def portfolio_performance(weights, mu, cov, riskfree):
    """
    Retorna (retorno_portfolio, volatilidade_portfolio, sharpe_portfolio)
    expected_returns: vetor de retornos esperados por ativo (numpy array)
    cov: matriz de covariância (numpy array)
    weights: vetor de pesos
    riskfree: taxa livre de risco (mesma periodicidade)
    """
    w = np.array(weights)
    ret = float(np.dot(w, mu))
    vol = float(np.sqrt(w.T @ cov @ w))
    if vol == 0:
        sharpe = 0.0
    else:
        sharpe = (ret - riskfree) / vol
    return ret, vol, sharpe

In [ ]:
riskfree = riskfree / 100 # Risk Free is indicated in percentage

In [ ]:
weights = [values / 100 for values in weightport] # Expected Returns are indicated in percentage

In [ ]:
# Convert covariance dataframe in numpy matrix
cov = covport.values

In [ ]:
# Extracting expected returns
mu = portfolio['RetE%'].values / 100

In [ ]:
# Porfolio Expected Return, Expected Risk and Expected Sharpe calculations
portfolioretexptotal, portfolioriskexptotal, portfoliosharpeexp = portfolio_performance(weights, mu, cov, riskfree)
# print (portfolioretexptotal, portfolioriskexptotal, portfoliosharpeexp)

In [ ]:
# Portfolio adding columns with Total Return, Total Risk and Sharpe values in first register(in the same line of IBOV value index). Other registers being filled with zero.
portfolio['RetET%'] = [portfolioretexptotal * 100] + [0] * (len(portfolio) - 1)
portfolio['RetET%'] = portfolio['RetET%'].round(1)
portfolio['RiskET%'] = [portfolioriskexptotal * 100] + [0] * (len(portfolio) - 1)
portfolio['RiskET%'] = portfolio['RiskET%'].round(0)
portfolio['SharpeE'] = [portfoliosharpeexp] + [0] * (len(portfolio) - 1)
portfolio['SharpeE'] = portfolio['SharpeE'].round(3)

In [ ]:
# Extracting real returns
mu = portfolio['RetH%'].values / 100

In [ ]:
# Porfolio Real (historic) Return, Risk and Sharpe calculations
portfoliorettotal, portfoliorisktotal, portfoliosharpe = portfolio_performance(weights, mu, cov, riskfree)
# print (portfoliorettotal, portfoliorisktotal, portfoliosharpe)

In [ ]:
portfolio['RetHT%'] = [portfoliorettotal * 100] + [0] * (len(portfolio) - 1)
portfolio['RetHT%'] = portfolio['RetHT%'].round(1)
portfolio['SharpeH'] = [portfoliosharpe] + [0] * (len(portfolio) - 1)
portfolio['SharpeH'] = portfolio['SharpeH'].round(3)

In [ ]:
# Calculate Total Beta and add in IBOV line
portfolio.at[0, 'Beta'] = (portfolio['W'] / 100 * portfolio['Beta']).sum()
portfolio['Beta'] = portfolio['Beta'].round(3)
# display(portfolio)

In [ ]:
# Portfolio MAXIMUM SHARPE calculation function
def max_sharpe_weights(mu, cov, risk_free, bounds=None, constraints=None):
    """
    Finds the weights that maximize the Sharpe ratio (using minimize on -sharpe).
    bounds: list of tuples (min, max) for each asset. Default: (0,1) for all.
    constraints: list of constraint dicts for scipy.optimize.minimize (optional).
    Returns the optimization result dictionary and the optimal weights.
    """
    n = len(mu)
    if bounds is None:
        bounds = tuple((0.0, 1.0) for _ in range(n))
    # constraint: sum of weights = 1
    cons = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}]
    if constraints:
        cons.extend(constraints)
    # initialization: equal weights
    x0 = np.ones(n) / n

    def neg_sharpe(w):
        _, _, s = portfolio_performance(w, mu, cov, risk_free)
        return -s

    res = minimize(neg_sharpe, x0, method='SLSQP', bounds=bounds, constraints=cons)
    return res, res.x if res.success else None


In [ ]:
# Extracting expected returns to calculte Sharpe Maximum
mu = portfolio['RetE%'].values / 100

In [ ]:
res_max, w_max = max_sharpe_weights(mu, cov, riskfree)
print("Max Sharpe:", portfolio_performance(w_max, mu, cov, riskfree)[2], "pesos:", w_max)
# print("Ret Max Sharpe:", w_max)

In [ ]:
#  Add Maximum Sharpe ticker weights to portfolio dataframe
portfolio['ShMaxE-W'] = w_max.round(3) * 100
# display(portfolio)

In [ ]:
# Add Maximum Sharpe value to portfolio dataframe in the first line
portfolio.at[0, 'ShMaxE-W'] = round(portfolio_performance(w_max, mu, cov, riskfree)[2], 3)
# display(portfolio)

In [ ]:
# Add Sharpe Maximum Return column and the value in the first line
portfolio['ShMaxRetE%'] = [portfolio_performance(w_max, mu, cov, riskfree)[0] *100] + [0] * (len(portfolio) - 1)
portfolio['ShMaxRetE%'] = portfolio['ShMaxRetE%'].round(1)
# display(portfolio)

In [ ]:
# Portfolio TARGET SHARPE calculation function (adjusted with the use of limitsport)
def weights_for_target_sharpe(mu, cov, target_sharpe, riskfree, limitsport, tol=1e-6, maxiter=1000):
    """
    Finds weights that approximate a target Sharpe ratio, using individual limits defined in limitsport.
    Minimizes the squared error between Sharpe(weights) and target_sharpe.
    """

    n = len(mu)

    # Build bounds using the limits provided by the user (limitsport)
    bounds = tuple((0.0, limitsport[i]) for i in range(n))

    # Constraint: sum of weights = 1
    cons = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}]

    # Initial guess
    x0 = np.ones(n) / n

    def sharpe_err_sq(w):
        _, _, s = portfolio_performance(w, mu, cov, riskfree)
        return (s - target_sharpe)**2

    opts = {'maxiter': maxiter, 'ftol': tol}
    res = minimize(sharpe_err_sq, x0, method='SLSQP', bounds=bounds, constraints=cons, options=opts)

    if res.success:
        ret, vol, s = portfolio_performance(res.x, mu, cov, riskfree)
        info = {'target_sharpe': target_sharpe, 'achieved_sharpe': s, 'ret': ret, 'vol': vol}
    else:
        info = {'message': res.message}

    return res, res.x if res.success else None, info

In [ ]:
# --- Define here your original list of limits in percentages (example you provided) ---
'''limitsport_percent = [0.0, 0.0, 15.0, 15.0, 15.0, 15.0, 5.0, 10.0, 10.0,
                      10.0, 5.0, 10.0, 15.0, 5.0, 8.0, 10.0, 5.0, 5.0,
                      15.0, 15.0]  # example: values in % (0..100)'''

# Convert to fraction (0..1)
limitsport = [float(x) / 100.0 for x in limitsassetsport]

# Check feasibility: the sum of the maximums must be >= 1 to allow sum(w)=1
sum_limits = sum(limitsport)
if sum_limits < 1.0 - 1e-12:
    # Strategy: scale the limits proportionally until the sum becomes 1.0
    # (keeping each limit <= 1.0). This preserves the ratio among limits.
    scale = 1.0 / sum_limits
    limitsport = [min(1.0, l * scale) for l in limitsport]
    print(f"WARNING: sum of limits < 1.0 — limits scaled proportionally (scale={scale:.6f}) to make the problem feasible.")
    # recompute sum for information
    print("New limits (fraction):", limitsport)
    print("Sum of new limits:", sum(limitsport))

# Define the target (your code)
# Target as a percentage of ShMaxE-W
target = portfolio.at[0, 'ShMaxE-W'] * target_per / 100
print("Target:", target)

# CORRECT call: pass limitsport (in fraction 0..1) as argument
res_tgt, w_tgt, info = weights_for_target_sharpe(mu, cov, target, riskfree, limitsport)

# Debug/output (optional)
# print("Target result:", info)
# print("Weights:", w_tgt)
# print("Achieved Sharpe:", info.get('achieved_sharpe'))

In [ ]:
#  Add Target Sharpe ticker weights to portfolio dataframe
portfolio['ShTg-W'] = (w_tgt * 100).round(3)
# Add Target (Achieved) Sharpe value to portfolio dataframe in the first line
portfolio.at[0, 'ShTg-W'] = round(info['achieved_sharpe'], 3)
# Add Target Maximum Return column and the value in the first line
portfolio['ShTgRetE%'] = [portfolio_performance(w_tgt, mu, cov, riskfree)[0] *100] + [0] * (len(portfolio) - 1)
portfolio['ShTgRetE%'] = portfolio['ShTgRetE%'].round(1)
display(portfolio)

In [ ]:
# Autentication in Google Docs (only once)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# Open workbook and worksheets
wb = gc.open('Quotes')
wss = wb.worksheet('Stockvar')
wsr = wb.worksheet('RealStatevar')
wssus = wb.worksheet('StockUSvar')
wseus = wb.worksheet('ETFUSvar')
wsrus = wb.worksheet('RealStateUSvar')
wsport = wb.worksheet('Portfolio')

In [ ]:
# Write data in the worksheets
wss.update([stockvar.columns.values.tolist()] + stockvar.values.tolist())
wsr.update([realstatevar.columns.values.tolist()] + realstatevar.values.tolist())
wssus.update([stockusvar.columns.values.tolist()] + stockusvar.values.tolist())
wseus.update([etfusvar.columns.values.tolist()] + etfusvar.values.tolist())
wsrus.update([reitusvar.columns.values.tolist()] + reitusvar.values.tolist())
wsport.update([portfolio.columns.values.tolist()] + portfolio.values.tolist())

In [ ]:
# B3 REAL STATE FUNDS EXPLORER ROUTINE
#
# example tickerr = ['IFIX.SA','BTLG11.SA','HGCR11.SA','HGBS11.SA','HGRE11.SA','HGRU11.SA','HSLG11.SA','HSML11.SA','HTMX11.SA','JSAF11.SA','JFLL11.SA','KNCA11.SA','KNHF11.SA','KNIP11.SA','MALL11.SA','MFII11.SA','SADI11.SA','TGAR11.SA','TRXF11.SA','VGHF11.SA','VISC11.SA']
# Remove ".SA" from real state funds dataframe tickers
tickerradjusted = [nome.replace('.SA', '') for nome in tickerr]
# Remove "IFIX" ticker
if "IFIX" in tickerradjusted:
    tickerradjusted.remove("IFIX")
#
# Funds list filter, exclude tickers not in real state funds list
fundsexplorer = fundsexplorer[fundsexplorer['Fundos'].isin(tickerradjusted)]
# display(fundsexplorer)
#
# Open worksheet
wsrfunds = wb.worksheet('FundsExplorer')
# Open worksheet
wsrfunds = wb.worksheet('FundsExplorer')

In [ ]:
# US STOCKS YAHOO FINANCE ROUTINE
#
# Example: tickersus = ['^GSPC','USDBRL=X','AAPL','AIG','BAC','RIO','DHI','EXC','KMB','KO','LOPE','LYB','MGA','MSFT','MSTR','NUE','NVDA','TGT','TMUS','UPS','UNH','XOM']
#
# Dictionary to store the data (each value is a dict for each ticker)
yfstockusdata = {}

# Set to accumulate all keys found across all tickers
all_keys = set()

for t in tickersus:
    try:
        tk = yf.Ticker(t)
        info = tk.info or {}
        # Ensures the ticker is always present as a field
        info_row = {'Ticker': t}
        # Add all key/value pairs returned by .info
        for k, v in info.items():
            info_row[k] = v
            all_keys.add(k)
        # Store the row
        yfstockusdata[t] = info_row
    except Exception as e:
        # In case of failure, register the ticker with only the 'Ticker' field
        # and leave the remaining keys absent (they will become NaN in the DataFrame)
        yfstockusdata[t] = {'Ticker': t}
        # (optional) you may log the error if needed:
        # print(f"Error for {t}: {e}")

# To guarantee ordered columns with 'Ticker' first,
# we explicitly build the list of columns
cols_other = sorted(all_keys - {'Ticker'})  # alphabetical sorting of other keys (optional)
cols_final = ['Ticker'] + cols_other

# Convert the dictionary to a DataFrame keeping numeric index (0,1,2,...)
# Note: using list(yfstockusdata.values()) to keep each dict as one row
yfstockus = pd.DataFrame(list(yfstockusdata.values()))

# Reindex columns to ensure 'Ticker' first and all remaining columns included
# Some keys might not appear in all tickers — this will not happen for missing keys,
# because all_keys was built from .info for each ticker.
# However, just to be safe:
existing_cols = [c for c in cols_final if c in yfstockus.columns]
yfstockus = yfstockus[existing_cols + [c for c in yfstockus.columns if c not in existing_cols]]

# Display the DataFrame (Jupyter/IPython)
# display(yfstockus)

In [ ]:
# Renaming ^GSPC to SP500 and USDBRLX to USDBRL
yfstockus['Ticker'] = yfstockus['Ticker'].str.replace('^GSPC', 'SP500', regex=False)
yfstockus['Ticker'] = yfstockus['Ticker'].str.replace('USDBRL=X', 'USDBRL', regex=False)
# display(yfstockus)

In [ ]:
# --- Value sanitization function ---
def sanitize_value(v, maxlen=500):
    # None / NaN
    if v is None:
        return ''
    # floats: reject NaN/Inf
    if isinstance(v, float):
        if not np.isfinite(v):
            return ''
        return float(v)
    # integers and booleans are OK
    if isinstance(v, (int, bool, np.integer, np.bool_)):
        return int(v) if isinstance(v, (int, np.integer)) else bool(v)
    # short strings are OK (truncate if too long)
    if isinstance(v, str):
        return v if len(v) <= maxlen else v[:maxlen]
    # time series / numpy types converted to truncated string
    # for all other types (dict, list, Timestamp, ndarray, Decimal, etc.)
    try:
        s = str(v)
        return s if len(s) <= maxlen else s[:maxlen]
    except Exception:
        return ''

# --- Sanitize column by column using Series.map (avoids applymap) ---
yfstockus_sanitized = yfstockus.copy()

for col in yfstockus_sanitized.columns:
    # Use map to apply the function in a vectorized way per column
    yfstockus_sanitized[col] = yfstockus_sanitized[col].map(lambda x: sanitize_value(x))

# --- Build the rows to send to Google Sheets ---
rows = [yfstockus_sanitized.columns.tolist()] + yfstockus_sanitized.values.tolist()

# --- Open/Create worksheet and update ---
try:
    wssyfstockus = wb.worksheet('YFStockUS')
except Exception as e:
    # if it does not exist, create it (adjust rows/cols if you want a specific size)
    wssyfstockus = wb.add_worksheet(title='YFStockUS', rows=str(len(rows)+10), cols=str(len(rows[0])+5))

# Perform the update (now with serializable data)
wssyfstockus.update(rows)